In [1]:
"""
Assignment 2, Task 1: Image Classification (Full Comparison)
- Compares two models: SimpleCNN and ResNet-18
- Compares two optimizers: SGD and Adam
- Runs all 4 combinations and prints a final report.
- Saves loss/accuracy plots for each run.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as T
import torchvision.models as models
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time
import os

# --- 0. Setup ---
NUM_EPOCHS = 10  # Keep this low for testing (e.g., 5-10)
BATCH_SIZE = 64
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# Create an output directory for plots
output_dir = "task_1_outputs"
os.makedirs(output_dir, exist_ok=True)

# --- 1. Data Loading (CIFAR-10) ---

transform = T.Compose([
    T.Resize((224, 224) if "ResNet" in "ResNet18" else (32, 32)), # ResNet needs 224x224
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Re-define transform for SimpleCNN
transform_simple = T.Compose([
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_dataset_simple = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_simple)
test_dataset_simple = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_simple)

class_names = train_dataset.classes



Using device: cuda


100%|██████████| 170M/170M [00:32<00:00, 5.23MB/s] 


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [2]:
# --- 2. Model Definitions ---

class SimpleCNN(nn.Module):
    """A simple baseline CNN for CIFAR-10 (32x32 images)."""
    def __init__(self, num_classes=10):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

def get_resnet_model(num_classes=10):
    """Loads a pre-trained ResNet-18 and adapts it for CIFAR-10."""
    model = models.resnet18(weights='IMAGENET1K_V1')
    for param in model.parameters():
        param.requires_grad = False
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_classes)
    return model

# --- 3. Training and Evaluation Loop ---

def train_model(model, model_name, optimizer_name, optimizer, train_loader, test_loader, criterion, num_epochs=NUM_EPOCHS):
    print(f"--- Training {model_name} with {optimizer_name} ---")
    
    history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}
    
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_train_loss = running_loss / len(train_loader)
        epoch_train_acc = 100 * correct / total
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc)

        # Validation
        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        all_preds, all_labels = [], []
        
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                
                all_preds.extend(predicted.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        epoch_val_loss = val_loss / len(test_loader)
        epoch_val_acc = 100 * correct / total
        history['val_loss'].append(epoch_val_loss)
        history['val_acc'].append(epoch_val_acc)
        
        print(f"Epoch {epoch+1}/{num_epochs} | "
              f"Train Loss: {epoch_train_loss:.4f} | Train Acc: {epoch_train_acc:.2f}% | "
              f"Val Loss: {epoch_val_loss:.4f} | Val Acc: {epoch_val_acc:.2f}%")

    print("Finished Training.")
    
    # Save plots
    plot_loss_accuracy(history, f"{model_name}_{optimizer_name}")
    plot_confusion_matrix(all_labels, all_preds, class_names, f"{model_name}_{optimizer_name}")
    
    return epoch_val_acc, history

# --- 4. Visualization Functions ---

def plot_loss_accuracy(history, run_name):
    """Plots and saves training and validation loss/accuracy curves."""
    epochs = range(1, len(history['train_loss']) + 1)
    plt.figure(figsize=(12, 5))
    
    plt.subplot(1, 2, 1)
    plt.plot(epochs, history['train_loss'], 'bo-', label='Training loss')
    plt.plot(epochs, history['val_loss'], 'ro-', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(epochs, history['train_acc'], 'bo-', label='Training acc')
    plt.plot(epochs, history['val_acc'], 'ro-', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    save_path = os.path.join(output_dir, f"{run_name}_metrics.png")
    plt.savefig(save_path)
    print(f"Saved metrics plot to {save_path}")
    plt.close()

def plot_confusion_matrix(all_labels, all_preds, class_names, run_name):
    """Plots and saves a confusion matrix."""
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix - {run_name}')
    
    save_path = os.path.join(output_dir, f"{run_name}_confusion_matrix.png")
    plt.savefig(save_path)
    print(f"Saved confusion matrix to {save_path}")
    plt.close()

# --- 5. Main Execution Block ---

if __name__ == "__main__":
    
    # Define all comparison configurations
    models_to_run = {
        "SimpleCNN": (SimpleCNN(num_classes=10), train_dataset_simple, test_dataset_simple),
        "ResNet18": (get_resnet_model(num_classes=10), train_dataset, test_dataset)
    }
    
    optimizers_to_run = {
        "SGD": (optim.SGD, {"lr": 0.01, "momentum": 0.9}),
        "Adam": (optim.Adam, {"lr": 0.001})
    }
    
    criterion = nn.CrossEntropyLoss()
    results = []

    for model_name, (model_instance, train_data, test_data) in models_to_run.items():
        
        train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
        test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)
        
        for opt_name, (opt_class, opt_params) in optimizers_to_run.items():
            
            # Re-initialize model and optimizer for a fresh run
            if model_name == "SimpleCNN":
                model = SimpleCNN(num_classes=10).to(DEVICE)
            else:
                model = get_resnet_model(num_classes=10).to(DEVICE)
                
            optimizer = opt_class(model.parameters(), **opt_params)
            
            start_time = time.time()
            final_acc, _ = train_model(
                model=model,
                model_name=model_name,
                optimizer_name=opt_name,
                optimizer=optimizer,
                train_loader=train_loader,
                test_loader=test_loader,
                criterion=criterion
            )
            end_time = time.time()
            
            results.append({
                "Model": model_name,
                "Optimizer": opt_name,
                "Final Val Accuracy": f"{final_acc:.2f}%",
                "Training Time (s)": f"{end_time - start_time:.2f}"
            })

    # --- 6. Final Report ---
    print("\n\n" + "="*50)
    print("      ASSIGNMENT 1 - FINAL COMPARISON REPORT")
    print("="*50)
    
    report_df = pd.DataFrame(results)
    print(report_df.to_string(index=False))
    
    print("\n" + "="*50)
    print(f"All plots saved to '{output_dir}' directory.")

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\alsto/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:01<00:00, 26.7MB/s]


--- Training SimpleCNN with SGD ---
Epoch 1/10 | Train Loss: 1.8338 | Train Acc: 31.94% | Val Loss: 1.5155 | Val Acc: 44.28%
Epoch 2/10 | Train Loss: 1.4030 | Train Acc: 48.91% | Val Loss: 1.2845 | Val Acc: 53.88%
Epoch 3/10 | Train Loss: 1.2357 | Train Acc: 55.88% | Val Loss: 1.1799 | Val Acc: 58.13%
Epoch 4/10 | Train Loss: 1.1321 | Train Acc: 59.90% | Val Loss: 1.1669 | Val Acc: 58.48%
Epoch 5/10 | Train Loss: 1.0590 | Train Acc: 62.46% | Val Loss: 1.1188 | Val Acc: 60.48%
Epoch 6/10 | Train Loss: 1.0020 | Train Acc: 64.49% | Val Loss: 1.1616 | Val Acc: 60.21%
Epoch 7/10 | Train Loss: 0.9473 | Train Acc: 66.16% | Val Loss: 1.1085 | Val Acc: 61.81%
Epoch 8/10 | Train Loss: 0.9004 | Train Acc: 67.99% | Val Loss: 1.0659 | Val Acc: 63.53%
Epoch 9/10 | Train Loss: 0.8673 | Train Acc: 69.17% | Val Loss: 1.0667 | Val Acc: 63.42%
Epoch 10/10 | Train Loss: 0.8209 | Train Acc: 70.74% | Val Loss: 1.1136 | Val Acc: 62.42%
Finished Training.
Saved metrics plot to task_1_outputs\SimpleCNN_SGD_met

Based on these results, it looks like the ResNet18 model is way better at this task than the SimpleCNN, getting a much higher validation accuracy around 78-81% compared to only 62-64%. But the trade-off is that the ResNet18 model took a lot longer to train, like over 1100 seconds, while the SimpleCNN was super fast at around 150 seconds. It also seems like for both models, the Adam optimizer worked a little better than SGD, giving us the best overall accuracy of 80.95% when used with ResNet18, though it did take slightly more time to train than SGD in both cases.